In [1]:
# если выдает ошибку с пакетами
# установить пакеты и перезапустить ядро
# TODO: сделать установку окружений

# !pip install scikit-learn --force
# !pip install catboost
# !pip install sktime==0.15.1
# !pip install pandas==1.3.5

In [1]:
# автоимпорт
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime
import os
import importlib

pd.set_option('display.max.columns', 300)

from core.calculator.storage import ModelDB
from core.calculator.core import ForecastConfig
from core.calculator.core import TrainingManager
from core.calculator.core import ForecastConfig
from core.calculator.core import ForecastEngine

from core.calculator.deposits import DepositsCalculationType
from core.calculator.deposits import DepositIterativeCalculator

from core.definitions import *
from core.project_update import load_portfolio

from core.models import DepositModels
from warnings import filterwarnings
filterwarnings('ignore')

## Подготовка сценария

In [150]:
# Дата из который мы прогнозируем
train_end = datetime(year=2024, month=3, day=31)

# Горизонт прогноза в месяцах
horizon = 12

In [151]:
# Данные для прогноза
scenario_data = {
    # Ожидаемый баланс на первую дату прогноза, задавать необязательно
     'expected_amount':      [np.nan for h in range(horizon)],
    # ССВ
     'SSV':                  [0.48 for h in range(horizon)],
    # ФОР
     'FOR':                  [4.5 for h in range(horizon)],
    # Трансфертные ставки
     'VTB_ftp_rate_[90d]':   [12.3 for h in range(horizon)],
     'VTB_ftp_rate_[180d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[365d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[548d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[730d]':  [12 for h in range(horizon)],
     'VTB_ftp_rate_[1095d]': [12 for h in range(horizon)],
    
    # Маржа бизнеса по срочностям
     'margin_[90d]':         [0.1 for h in range(horizon)],
     'margin_[180d]':        [0.1 for h in range(horizon)],
     'margin_[365d]':        [0.1 for h in range(horizon)],
     'margin_[548d]':        [0.1 for h in range(horizon)],
     'margin_[730d]':        [0.2 for h in range(horizon)],
     'margin_[1095d]':       [0.2 for h in range(horizon)],
    
    # Спред Привилегия - Массовый (на сколько в среднем ставки по сегменту Привилегия больше чем ставки по массовому сегменту)
     'priv_spread':          [0.4 for h in range(horizon)],
    # Спред ВИП - Массовый (на сколько в среднем ставки по сегменту ВИП больше чем ставки по массовому сегменту)
     'vip_spread':           [0.8 for h in range(horizon)],
    
    # Ниже три спреда по разным типам опциональности по отношению к безопциональным вкладам (Подразумевается, что они, как правило, отрицательные)
    # r - возможности пополнения, s - возможность расходных операций
    
    # На сколько ставка по расходным вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r0s1_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s0_spread':          [-1 for h in range(horizon)],
    
    # На сколько ставка по расходно-пополняемым вкладам выше чем ставка по безопциональным вкладам (Если ниже - то со знаком минус)
     'r1s1_spread':          [-1.2 for h in range(horizon)],
    
    # Ставка по лучшему предложению сбера
     'SBER_max_rate':        [11.2 for h in range(horizon)],
    
    # Базовая ставка по НС
     'SA_rate':              [5 for h in range(horizon)]
}
scenario_df_user = pd.DataFrame(scenario_data)

In [152]:
scenario_df = preprocess_scenario(scenario_df_user, train_end, horizon)

In [153]:
scenario_df

,expected_amount,SBER_max_rate,rate_sa_weighted,VTB_ftp_rate_[90d],VTB_ftp_rate_[180d],VTB_ftp_rate_[365d],VTB_ftp_rate_[548d],VTB_ftp_rate_[730d],VTB_ftp_rate_[1095d],VTB_weighted_rate_[mass]_[r0s0]_[90d],VTB_weighted_rate_[mass]_[r0s0]_[180d],VTB_weighted_rate_[mass]_[r0s0]_[365d],VTB_weighted_rate_[mass]_[r0s0]_[548d],VTB_weighted_rate_[mass]_[r0s0]_[730d],VTB_weighted_rate_[mass]_[r0s0]_[1095d],VTB_weighted_rate_[mass]_[r0s1]_[90d],VTB_weighted_rate_[mass]_[r0s1]_[180d],VTB_weighted_rate_[mass]_[r0s1]_[365d],VTB_weighted_rate_[mass]_[r0s1]_[548d],VTB_weighted_rate_[mass]_[r0s1]_[730d],VTB_weighted_rate_[mass]_[r0s1]_[1095d],VTB_weighted_rate_[mass]_[r1s0]_[90d],VTB_weighted_rate_[mass]_[r1s0]_[180d],VTB_weighted_rate_[mass]_[r1s0]_[365d],VTB_weighted_rate_[mass]_[r1s0]_[548d],VTB_weighted_rate_[mass]_[r1s0]_[730d],VTB_weighted_rate_[mass]_[r1s0]_[1095d],VTB_weighted_rate_[mass]_[r1s1]_[90d],VTB_weighted_rate_[mass]_[r1s1]_[180d],VTB_weighted_rate_[mass]_[r1s1]_[365d],VTB_weighted_rate_[mass]_[r1s1]_[548d],VTB_weighted_rate_[mass]_[r1s1]_[730d],VTB_weighted_rate_[mass]_[r1s1]_[1095d],VTB_weighted_rate_[priv]_[r0s0]_[90d],VTB_weighted_rate_[priv]_[r0s0]_[180d],VTB_weighted_rate_[priv]_[r0s0]_[365d],VTB_weighted_rate_[priv]_[r0s0]_[548d],VTB_weighted_rate_[priv]_[r0s0]_[730d],VTB_weighted_rate_[priv]_[r0s0]_[1095d],VTB_weighted_rate_[priv]_[r0s1]_[90d],VTB_weighted_rate_[priv]_[r0s1]_[180d],VTB_weighted_rate_[priv]_[r0s1]_[365d],VTB_weighted_rate_[priv]_[r0s1]_[548d],VTB_weighted_rate_[priv]_[r0s1]_[730d],VTB_weighted_rate_[priv]_[r0s1]_[1095d],VTB_weighted_rate_[priv]_[r1s0]_[90d],VTB_weighted_rate_[priv]_[r1s0]_[180d],VTB_weighted_rate_[priv]_[r1s0]_[365d],VTB_weighted_rate_[priv]_[r1s0]_[548d],VTB_weighted_rate_[priv]_[r1s0]_[730d],VTB_weighted_rate_[priv]_[r1s0]_[1095d],VTB_weighted_rate_[priv]_[r1s1]_[90d],VTB_weighted_rate_[priv]_[r1s1]_[180d],VTB_weighted_rate_[priv]_[r1s1]_[365d],VTB_weighted_rate_[priv]_[r1s1]_[548d],VTB_weighted_rate_[priv]_[r1s1]_[730d],VTB_weighted_rate_[priv]_[r1s1]_[1095d],VTB_weighted_rate_[vip]_[r0s0]_[90d],VTB_weighted_rate_[vip]_[r0s0]_[180d],VTB_weighted_rate_[vip]_[r0s0]_[365d],VTB_weighted_rate_[vip]_[r0s0]_[548d],VTB_weighted_rate_[vip]_[r0s0]_[730d],VTB_weighted_rate_[vip]_[r0s0]_[1095d],VTB_weighted_rate_[vip]_[r0s1]_[90d],VTB_weighted_rate_[vip]_[r0s1]_[180d],VTB_weighted_rate_[vip]_[r0s1]_[365d],VTB_weighted_rate_[vip]_[r0s1]_[548d],VTB_weighted_rate_[vip]_[r0s1]_[730d],VTB_weighted_rate_[vip]_[r0s1]_[1095d],VTB_weighted_rate_[vip]_[r1s0]_[90d],VTB_weighted_rate_[vip]_[r1s0]_[180d],VTB_weighted_rate_[vip]_[r1s0]_[365d],VTB_weighted_rate_[vip]_[r1s0]_[548d],VTB_weighted_rate_[vip]_[r1s0]_[730d],VTB_weighted_rate_[vip]_[r1s0]_[1095d],VTB_weighted_rate_[vip]_[r1s1]_[90d],VTB_weighted_rate_[vip]_[r1s1]_[180d],VTB_weighted_rate_[vip]_[r1s1]_[365d],VTB_weighted_rate_[vip]_[r1s1]_[548d],VTB_weighted_rate_[vip]_[r1s1]_[730d],VTB_weighted_rate_[vip]_[r1s1]_[1095d]
2024-04-30,NaN,11.2,5.0,12.3,12.0,12.0,12.0,12.0,12.0,10.3665,10.08,10.08,10.08,9.98,9.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.1665,8.88,8.88,8.88,8.78,8.78,10.7665,10.48,10.48,10.48,10.38,10.38,9.7665,9.48,9.48,9.48,9.38,9.38,9.7665,9.48,9.48,9.48,9.38,9.38,9.5665,9.28,9.28,9.28,9.18,9.18,11.1665,10.88,10.88,10.88,10.78,10.78,10.1665,9.88,9.88,9.88,9.78,9.78,10.1665,9.88,9.88,9.88,9.78,9.78,9.9665,9.68,9.68,9.68,9.58,9.58
2024-05-31,NaN,11.2,5.0,12.3,12.0,12.0,12.0,12.0,12.0,10.3665,10.08,10.08,10.08,9.98,9.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.1665,8.88,8.88,8.88,8.78,8.78,10.7665,10.48,10.48,10.48,10.38,10.38,9.7665,9.48,9.48,9.48,9.38,9.38,9.7665,9.48,9.48,9.48,9.38,9.38,9.5665,9.28,9.28,9.28,9.18,9.18,11.1665,10.88,10.88,10.88,10.78,10.78,10.1665,9.88,9.88,9.88,9.78,9.78,10.1665,9.88,9.88,9.88,9.78,9.78,9.9665,9.68,9.68,9.68,9.58,9.58
2024-06-30,NaN,11.2,5.0,12.3,12.0,12.0,12.0,12.0,12.0,10.3665,10.08,10.08,10.08,9.98,9.98,9.3665,9.08,9.08,9.08,8.98,8.98,9.3665,9.08,9.

Если судить по атрибутному составу, то можем без доп рассчетов взять данные со сценариев

По сути нужно запарсить: 
* маскимальную ставку сбера (ставку ЦБ)  
* трансфертные ставки для продуктов 
* ставки в разрезах:
    * Сегмент
    * опциональность
    * срочность

Опциональности:  
r - буква, которая означает возможность пополнения вкладов (r1 - опция есть, r0 - опции нет). Происхождение: r от слова replenishable  
s - буква, которая означает возможность расходных операций вкладов (s1 - опция есть, s0 - опции нет). Происхождение: s от слова subtraction  

## Создаем парсер сценариев

### Чтение данных

In [159]:
def parse_scenario(scenario, scenario_df, sber_rate, sa_rate):
    """
    Парсим все, НС и макс ставку Сбера задаем сами
    scenario - спарсенные данные 
    scenario_df -  датафрейм с индексами и колонками
    
    """
    
    # FTP ставки
    ftp_rates_list = ['VTB_ftp_rate_[90d]', 
                  'VTB_ftp_rate_[180d]', 
                  'VTB_ftp_rate_[365d]',
                  'VTB_ftp_rate_[548d]', 
                  'VTB_ftp_rate_[730d]', 
                  'VTB_ftp_rate_[1095d]']
    
    
    scenario_res = pd.DataFrame(columns=scenario_df.columns)
    
    for i, ftp_name in enumerate(ftp_rates_list):

        scenario_res.loc[0, ftp_name] = scenario.iloc[14, 4+i]*100
        
        
        
    segm_dict = {'mass': 70,
                'priv': 48,
                'vip': 27}
    
    def get_segm_values(df, segm_dict): 
        for segm in segm_dict.keys():

            row = segm_dict[segm]

            # без опций
            list_rates = list_rates_gen(segm, '0', '0')
            for i, rate_name in enumerate(list_rates):
                df.loc[0, rate_name] = scenario.iloc[row, 4+i] * 100

            # с пополнением
            list_rates = list_rates_gen(segm, '1', '0')
            for i, rate_name in enumerate(list_rates):
                df.loc[0, rate_name] = scenario.iloc[row, 20+i] * 100

            # с пополнением и снятием
            list_rates = list_rates_gen(segm, '1', '1')
            for i, rate_name in enumerate(list_rates):
                df.loc[0, rate_name] = scenario.iloc[row, 28+i] * 100

            # со снятием
            list_rates = list_rates_gen(segm, '0', '1')
            # заполняем заглушками
            for i, rate_name in enumerate(list_rates):
                df.loc[0, rate_name] = 0.001

        return df
    
    
    def list_rates_gen(segm, repl, sub):

        list_rates = [f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[90d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[180d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[365d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[548d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[730d]',
                        f'VTB_weighted_rate_[{segm}]_[r{repl}s{sub}]_[1095d]']

        return list_rates
    
    scenario_res = get_segm_values(scenario_res, segm_dict)
    
    scenario_res['SBER_max_rate'] = sber_rate
    scenario_res['rate_sa_weighted'] = sa_rate
    
    
    scenario_res = scenario_res.append([scenario_res]*11, ignore_index=True)
    scenario_res.index = scenario_df.index
    
    
    return scenario_res

In [178]:
# считанный сценарий
sc1 = pd.read_excel('04_24_scenarios.xlsx', sheet_name='Сценарий 1')
sc2 = pd.read_excel('04_24_scenarios.xlsx', sheet_name='Сценарий 2')
sc3 = pd.read_excel('04_24_scenarios.xlsx', sheet_name='Сценарий 3')

In [183]:
scenario_1 = parse_scenario(sc1, scenario_df, sber_rate=16, sa_rate=11)
scenario_2 = parse_scenario(sc2, scenario_df, sber_rate=15, sa_rate=11)
scenario_3 = parse_scenario(sc3, scenario_df, sber_rate=17, sa_rate=11)

## Сохранить сценарии

In [184]:
# для записи и чтения экселя
import pip
pip.main(['install', 'openpyxl'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://nxrm:8081/repository/pypi/simple


0

In [185]:
import openpyxl

In [186]:
with pd.ExcelWriter("04_24_scenarious.xlsx") as writer:
    scenario_1.to_excel(writer, sheet_name='scenario_1', index=True)
    scenario_2.to_excel(writer, sheet_name='scenario_2', index=True)
    scenario_3.to_excel(writer, sheet_name='scenario_3', index=True)